<a href="https://colab.research.google.com/github/tonycota/deep-learning-challenge/blob/main/optimized_notebooks/deep_learning_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# FIRST ATTEMPT AT OPTIMIZATION

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME' and AFFILIATION/ STATUS/ ASK_AMT columns
columns_to_drop = ['EIN', 'NAME', 'AFFILIATION', 'STATUS', 'ASK_AMT']
application_df.drop(columns=columns_to_drop, inplace=True)
application_df.head(10)

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,C1000,ProductDev,Association,0,N,1
1,T3,C2000,Preservation,Co-operative,1-9999,N,1
2,T5,C3000,ProductDev,Association,0,N,0
3,T3,C2000,Preservation,Trust,10000-24999,N,1
4,T3,C1000,Heathcare,Trust,100000-499999,N,1
5,T3,C1200,Preservation,Trust,0,N,1
6,T3,C1000,Preservation,Trust,100000-499999,N,1
7,T3,C2000,Preservation,Trust,10M-50M,N,1
8,T7,C1000,ProductDev,Trust,1-9999,N,1
9,T5,C3000,ProductDev,Association,0,N,0


In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE          17
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
IS_SUCCESSFUL              2
dtype: int64

In [4]:
print(application_df.value_counts())

APPLICATION_TYPE  CLASSIFICATION  USE_CASE      ORGANIZATION  INCOME_AMT     SPECIAL_CONSIDERATIONS  IS_SUCCESSFUL
T3                C1000           Preservation  Trust         0              N                       1                4014
                                                                                                     0                2495
                  C2000           Preservation  Trust         0              N                       1                1361
                                                                                                     0                1235
                  C1200           Preservation  Association   0              N                       0                1184
                                                                                                                      ... 
                  C2500           Preservation  Trust         0              N                       1                   1
                  C2561 

In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_counts = application_df["APPLICATION_TYPE"].value_counts()
app_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classy_counts = application_df["CLASSIFICATION"].value_counts()
classy_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
helpful_classy_counts = classy_counts.loc[classy_counts > 1]
helpful_classy_counts[:15]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classy_counts[classy_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
app_numeric = pd.get_dummies(application_df)
app_numeric

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,0,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,0,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [11]:
# Split our preprocessed data into our features and target arrays
X = app_numeric.drop('IS_SUCCESSFUL', axis=1)
y = app_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  15
hidden_nodes_layer2 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                540       
                                                                 
 dense_1 (Dense)             (None, 15)                240       
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 796 (3.11 KB)
Trainable params: 796 (3.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6306 - accuracy: 0.6370
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6150 - accuracy: 0.6485
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6124 - accuracy: 0.6520
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6110 - accuracy: 0.6543
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6099 - accuracy: 0.6545
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6093 - accuracy: 0.6558
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6087 - accuracy: 0.6544
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6084 - accuracy: 0.6554
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6078 - accuracy: 0.6559
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6073 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6122 - accuracy: 0.6525 - 590ms/epoch - 2ms/step
Loss: 0.612170934677124, Accuracy: 0.6524781584739685


In [17]:
# Export our model to HDF5 file
# Working in google colab so the code gets interesting here
# import os dependency
import os
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
file_path = os.path.join(desktop_path, "AlphabetSoupCharity_1.h5")
nn.save(file_path)

# save file from google colab
from google.colab import files
files.download(file_path)
# Ensure file was saved into correct directory
print(f"Model saved to {file_path}")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model saved to /root/Desktop/AlphabetSoupCharity_1.h5


In [ ]:
# This attempt at optimization did NOT work the accuracy score went down to 65%